# 뉴스 편향성 분석 모델

이 노트북은 뉴스 기사의 민주당과 국힘에 대한 편향성을 분석하는 딥러닝 모델을 구현합니다.

In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    DataCollatorWithPadding
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1. 데이터 로드 및 전처리

In [3]:
# 베이스라인 결과 로드
baseline_results = pd.read_csv('/content/drive/MyDrive/텍스트데이터분석을 위한 딥러닝/팀프로젝트/baseline_results/baseline_results.csv').iloc[0]

In [ ]:
# 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/텍스트데이터분석을 위한 딥러닝/팀프로젝트/data/정당_관점_라벨링_최종_업데이트.csv')  # 파일 경로는 실제 경로에 맞게 수정해주세요

# 정당 레이블 매핑
party_mapping = {'국민의힘': 0, '민주당': 1, '그외': 2}

# 정당 레이블 변환
df['party_label'] = df['party'].map(party_mapping)

# 감정 레이블 매핑
sentiment_mapping = {'긍정': 0, '중립': 1, '부정': 2}

# 감정 레이블 변환
df['sentiment_label'] = df['sentiment'].map(sentiment_mapping)

# NaN 값 처리
df = df.dropna(subset=['title_cleaned', 'content_cleaned', 'party_label', 'sentiment_label'])

# 제목과 본문 결합
df['text'] = df['title_cleaned'] + ' ' + df['content_cleaned']

print(f"전체 데이터 수: {len(df)}")
print("\n정당별 기사 수:")
print(df['party'].value_counts())
print("\n정당 레이블 분포:")
print(df['party_label'].value_counts())
print("\n감정별 기사 수:")
print(df['sentiment'].value_counts())
print("\n감정 레이블 분포:")
print(df['sentiment_label'].value_counts())

## 2. 데이터셋 클래스 정의

In [5]:
class NewsDataset(Dataset):
    def __init__(self, texts, party_labels, sentiment_labels, tokenizer, max_length=512):
        self.texts = texts
        self.party_labels = party_labels
        self.sentiment_labels = sentiment_labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # 텐서 차원 조정
        input_ids = encoding['input_ids'].squeeze(0)  # [max_length]
        attention_mask = encoding['attention_mask'].squeeze(0)  # [max_length]

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'party_label': torch.tensor(self.party_labels[idx], dtype=torch.long),
            'sentiment_label': torch.tensor(self.sentiment_labels[idx], dtype=torch.long)
        }

## 3. 모델 정의

In [6]:
class NewsBiasModel(torch.nn.Module):
    def __init__(self, model_name, num_party_labels=3, num_sentiment_labels=3,
                 class_weights=None, dropout_rate=0.2, hidden_size=512):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.register_buffer("class_weights", torch.tensor(class_weights, dtype=torch.float))

        hidden_size_bert = self.bert.config.hidden_size
        self.dropout = torch.nn.Dropout(dropout_rate)

        # 특성 추출 레이어
        self.feature_layer = torch.nn.Sequential(
            torch.nn.Linear(hidden_size_bert, hidden_size),
            torch.nn.LayerNorm(hidden_size),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout_rate)
        )

        # BiLSTM 레이어
        self.bilstm = torch.nn.LSTM(
            input_size=hidden_size,
            hidden_size=hidden_size // 2,
            num_layers=1,
            batch_first=True,
            bidirectional=True,
            dropout=dropout_rate + 0.1
        )

        # 정당 분류기
        self.party_classifier = torch.nn.Sequential(
            torch.nn.Linear(hidden_size, hidden_size // 2),
            torch.nn.LayerNorm(hidden_size // 2),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout_rate),
            torch.nn.Linear(hidden_size // 2, num_party_labels)
        )

        # 감성 분류기
        self.sentiment_classifier = torch.nn.Sequential(
            torch.nn.Linear(hidden_size, hidden_size // 2),
            torch.nn.LayerNorm(hidden_size // 2),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout_rate),
            torch.nn.Linear(hidden_size // 2, num_sentiment_labels)
        )

    def forward(self, input_ids, attention_mask, party_label=None, sentiment_label=None):
        # BERT 출력
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state

        # 특성 추출
        features = self.dropout(sequence_output)
        features = self.feature_layer(features) 

        # BiLSTM 적용
        lstm_output, _ = self.bilstm(features)

        # 최종 특성 추출 (평균 풀링)
        final_features = torch.mean(lstm_output, dim=1)

        # 예측
        party_logits = self.party_classifier(final_features)
        sentiment_logits = self.sentiment_classifier(final_features)

        output = {
            'party_logits': party_logits,
            'sentiment_logits': sentiment_logits
        }

        if party_label is not None and sentiment_label is not None:
            loss_fct_party = torch.nn.CrossEntropyLoss(weight=self.class_weights)
            loss_fct_sentiment = torch.nn.CrossEntropyLoss()
            party_loss = loss_fct_party(party_logits, party_label)
            sentiment_loss = loss_fct_sentiment(sentiment_logits, sentiment_label)
            output['loss'] = (party_loss + sentiment_loss) / 2

        return output

## 4. 학습 준비

In [7]:
# 데이터 분할
train_texts, val_texts, train_party_labels, val_party_labels, train_sentiment_labels, val_sentiment_labels = train_test_split(
    df['text'].values,
    df['party_label'].values,
    df['sentiment_label'].values,
    test_size=0.2,
    random_state=42
)

In [ ]:
# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

# 데이터셋 생성
train_dataset = NewsDataset(train_texts, train_party_labels, train_sentiment_labels, tokenizer)
val_dataset = NewsDataset(val_texts, val_party_labels, val_sentiment_labels, tokenizer)

In [ ]:
label_list = [example['party_label'] for example in train_dataset]
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array([0, 1, 2]),  # 0: 국민의힘, 1: 민주당, 2: 그외
    y=np.array(label_list)
)
print("클래스 가중치:", class_weights)

In [ ]:
# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화 및 GPU 이동을 분리
model = NewsBiasModel('klue/roberta-base', class_weights=class_weights)
model = model.to(device)

# 커스텀 데이터 콜레이터
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
def compute_metrics(eval_pred):
    party_preds = eval_pred.predictions
    party_labels = eval_pred.label_ids
    sentiment_preds = eval_pred.predictions
    sentiment_labels = eval_pred.label_ids

    party_preds = np.argmax(party_preds, axis=1)
    sentiment_preds = np.argmax(sentiment_preds, axis=1)

    party_report = classification_report(
        party_labels,
        party_preds,
        target_names=['국민의힘', '민주당', '그외'],
        output_dict=True,
        zero_division=0
    )

    sentiment_report = classification_report(
        sentiment_labels,
        sentiment_preds,
        target_names=['긍정', '중립', '부정'],
        output_dict=True,
        zero_division=0
    )

    return {
        'party_f1': party_report['weighted avg']['f1-score'],
        'party_accuracy': party_report['accuracy'],
        'sentiment_f1': sentiment_report['weighted avg']['f1-score'],
        'sentiment_accuracy': sentiment_report['accuracy'],
    }

In [ ]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./news_bias_results',  # 디렉토리 이름 변경
    num_train_epochs=9,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=3.3541e-05,
    warmup_ratio=0.1135,
    weight_decay=0.0256,
    logging_dir='./news_bias_logs',  # 로그 디렉토리 이름 변경
    logging_steps=100,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='party_f1',  # 평가 메트릭 이름 변경
    gradient_accumulation_steps=3,  # 그래디언트 누적
    max_grad_norm=0.7155,  # 그래디언트 클리핑
    fp16=True,  # 혼합 정밀도 학습
    label_smoothing_factor=0.0672,  # 레이블 스무딩
    optim='adamw_torch',  # AdamW 옵티마이저 사용
    lr_scheduler_type='cosine'
)

# 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.001)]
)

## 5. 학습 및 평가

In [ ]:
# 학습 실행
trainer.train()

# 최종 평가
final_metrics = trainer.evaluate()
print("\n최종 평가 결과:")
print(f"정당 분류 F1 점수: {final_metrics['eval_party_f1']:.4f}")
print(f"정당 분류 정확도: {final_metrics['eval_party_accuracy']:.4f}")
print(f"감정 분류 F1 점수: {final_metrics['eval_sentiment_f1']:.4f}")
print(f"감정 분류 정확도: {final_metrics['eval_sentiment_accuracy']:.4f}")

In [14]:
# 베이스라인과 성능 비교
comparison_results = {
    'Model': ['Baseline', 'News Bias BERT'],  # 모델 이름 변경
    'Party F1': [baseline_results['party_f1'], final_metrics['eval_party_f1']],  # 메트릭 이름 변경
    'Party Accuracy': [baseline_results['party_accuracy'], final_metrics['eval_party_accuracy']],  # 메트릭 이름 변경
}

comparison_df = pd.DataFrame(comparison_results)
comparison_df.to_csv('./news_bias_results/model_comparison.csv', index=False)  # 저장 경로 변경
comparison_df.head()

,Model,Party F1,Party Accuracy
0,Baseline,0.640569,0.643902
1,News Bias BERT,0.737795,0.737864


In [ ]:
# 성능 향상 시각화
plt.figure(figsize=(12, 6))
metrics = ['Party F1', 'Party Accuracy']  # 메트릭 이름 변경
x = np.arange(len(metrics))
width = 0.35

plt.bar(x - width/2, comparison_df.iloc[0, 1:], width, label='Baseline')
plt.bar(x + width/2, comparison_df.iloc[1, 1:], width, label='News Bias BERT')  # 모델 이름 변경

plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Model Performance Comparison')
plt.xticks(x, metrics)
plt.legend()

plt.savefig('./news_bias_results/performance_comparison.png')  # 저장 경로 변경
plt.show()

## 6. 하이퍼파라미터 튜닝

In [ ]:
!pip install optuna
!pip install optuna-integration[pytorch_lightning]  # 설치 후 세션을 다시 시작해야 함

In [15]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from sklearn.model_selection import KFold

In [23]:
def objective(trial):
    # 하이퍼파라미터 정의
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-6, 1e-4, log=True),
        'weight_decay': trial.suggest_float('weight_decay', 0.01, 0.04),
        'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.3),
        'hidden_size': trial.suggest_categorical('hidden_size', [256, 384, 512, 768, 1024]),
        'num_epochs': trial.suggest_int('num_epochs', 5, 15),
        'batch_size': trial.suggest_categorical('batch_size', [16, 32, 64]),
        'warmup_ratio': trial.suggest_float('warmup_ratio', 0.1, 0.2),
        'label_smoothing': trial.suggest_float('label_smoothing', 0.05, 0.10),
        'gradient_accumulation_steps': trial.suggest_int('gradient_accumulation_steps', 1, 4),
        'max_grad_norm': trial.suggest_float('max_grad_norm', 0.5, 1.0)
    }

    # K-fold 교차 검증
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
        # 데이터 분할
        train_texts = df.iloc[train_idx]['text'].values
        val_texts = df.iloc[val_idx]['text'].values
        train_labels = df.iloc[train_idx]['party_label'].values
        val_labels = df.iloc[val_idx]['party_label'].values

        # 데이터셋 생성
        train_dataset = NewsDataset(train_texts, train_labels, tokenizer)
        val_dataset = NewsDataset(val_texts, val_labels, tokenizer)

        # 모델 초기화
        model = NewsBiasModel(
            'klue/roberta-base',
            class_weights=class_weights,
            dropout_rate=params['dropout_rate'],
            hidden_size=params['hidden_size']
        )
        model = model.to(device)

        # 학습 인자 설정
        training_args = TrainingArguments(
            output_dir='./temp_results',
            num_train_epochs=params['num_epochs'],
            per_device_train_batch_size=params['batch_size'],
            per_device_eval_batch_size=params['batch_size'] * 2,
            learning_rate=params['learning_rate'],
            warmup_ratio=params['warmup_ratio'],
            weight_decay=params['weight_decay'],
            logging_dir='./temp_logs',
            logging_steps=100,
            eval_strategy='epoch',
            save_strategy='no',
            load_best_model_at_end=False,
            save_total_limit=0,
            metric_for_best_model='party_f1',
            gradient_accumulation_steps=params['gradient_accumulation_steps'],
            max_grad_norm=params['max_grad_norm'],
            fp16=True,
            label_smoothing_factor=params['label_smoothing'],
            optim='adamw_torch',
            lr_scheduler_type='cosine'
        )

        # 트레이너 초기화
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
            compute_metrics=compute_metrics,
            callbacks=[
                EarlyStoppingCallback(
                    early_stopping_patience=3,
                    early_stopping_threshold=0.001
                )
            ]
        )

        # 학습
        trainer.train()

        # 평가
        metrics = trainer.evaluate()
        scores.append(metrics['eval_party_f1'])

    # 평균 F1 점수 반환
    return np.mean(scores)

In [ ]:
# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # 20회 시도

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

In [26]:
# 모든 trial의 결과를 DataFrame으로 저장
trials_df = study.trials_dataframe()
trials_df.to_csv('hyperparameter_trials.csv', index=False)
trials_df.head()

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_dropout_rate,params_gradient_accumulation_steps,params_hidden_size,params_label_smoothing,params_learning_rate,params_max_grad_norm,params_num_epochs,params_warmup_ratio,params_weight_decay,state
0,0,0.663819,2025-06-14 11:46:07.871629,2025-06-14 11:51:07.974238,0 days 00:05:00.102609,16,0.279187,1,384,0.065000,0.000015,0.659867,12,0.163519,0.033839,COMPLETE
1,1,0.569027,2025-06-14 11:51:07.975207,2025-06-14 11:58:06.066161,0 days 00:06:58.090954,16,0.158626,3,512,0.098760,0.000002,0.505384,13,0.154662,0.035720,COMPLETE
2,2,0.674118,2025-06-14 11:58:06.067150,2025-06-14 12:03:53.722856,0 days 00:05:47.655706,64,0.118109,4,768,0.073240,0.000029,0.503510,14,0.132069,0.020156,COMPLETE
3,3,0.692987,2025-06-14 12:03:53.723790,2025-06-14 12:06:50.511882,0 days 00:02:56.788092,32,0.260503,1,256,0.052638,0.000017,0.624953,6,0.167738,0.012483,COMPLETE
4,4,0.426354,2025-06-14 12:06:50.512778,2025-06-14 12:09:59.830864,0 days 00:03:09.318086,64,0.162907,1,256,0.054680,0.000002,0.777909,7,0.157970,0.038072,COMPLETE


In [27]:
print(trials_df.sort_values(by='value', ascending=False).iloc[0])

number                                                         3
value                                                   0.692987
datetime_start                        2025-06-14 12:03:53.723790
datetime_complete                     2025-06-14 12:06:50.511882
duration                                  0 days 00:02:56.788092
params_batch_size                                             32
params_dropout_rate                                     0.260503
params_gradient_accumulation_steps                             1
params_hidden_size                                           256
params_label_smoothing                                  0.052638
params_learning_rate                                    0.000017
params_max_grad_norm                                    0.624953
params_num_epochs                                              6
params_warmup_ratio                                     0.167738
params_weight_decay                                     0.012483
state                    

In [28]:
# 최적의 하이퍼파라미터로 모델 학습
best_params = trial.params

In [29]:
best_params

{'learning_rate': 1.676253496408822e-05,
 'weight_decay': 0.012483159054831302,
 'dropout_rate': 0.2605034927659957,
 'hidden_size': 256,
 'num_epochs': 6,
 'batch_size': 32,
 'warmup_ratio': 0.16773817701335603,
 'label_smoothing': 0.05263779406124121,
 'gradient_accumulation_steps': 1,
 'max_grad_norm': 0.6249529006556371}

## 7. 모델 저장 및 로드

In [ ]:
# 모델 저장
trainer.save_model('./news_bias_model')  # 저장 경로 변경
tokenizer.save_pretrained('./news_bias_model')  # 저장 경로 변경

## 8. 새로운 기사에 대한 예측

In [29]:
# 모델 로드
def load_model(model_path, class_weights):
    model = NewsBiasModel('klue/roberta-base', class_weights=class_weights)  # 모델 클래스 변경

    # safetensors 파일 로드
    from safetensors.torch import load_file
    model.load_state_dict(load_file(f'{model_path}/model.safetensors'))

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer

In [17]:
def predict_news(text, model, tokenizer, device):
    model.eval()
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        party_pred = torch.argmax(outputs['party_logits'], dim=1).item()
        sentiment_pred = torch.argmax(outputs['sentiment_logits'], dim=1).item()

    # 정당 레이블 매핑
    party_mapping = {0: '국민의힘', 1: '민주당', 2: '그외'}
    sentiment_mapping = {0: '긍정', 1: '중립', 2: '부정'}
    
    return party_mapping[party_pred], sentiment_mapping[sentiment_pred]

In [20]:
def predict_csv_file(csv_path, model, tokenizer, device):
    # CSV 파일 로드
    df = pd.read_csv(csv_path)

    # 예측 결과를 저장할 리스트
    party_predictions = []
    sentiment_predictions = []

    # 각 텍스트에 대해 예측 수행
    for text in tqdm(df['text'], desc="Predicting"):
        party_pred, sentiment_pred = predict_news(text, model, tokenizer, device)
        party_predictions.append(party_pred)
        sentiment_predictions.append(sentiment_pred)

    # 예측 결과를 DataFrame에 추가
    df['party'] = party_predictions
    df['sentiment'] = sentiment_predictions

    # # 결과 저장
    # output_path = csv_path.replace('.csv', '_predicted.csv')
    # df.to_csv(output_path, index=False)

    # 예측 결과 통계 출력
    print(f"\n{csv_path} 예측 결과:")
    print("\n정당 예측 분포:")
    print(df['party'].value_counts())
    print("\n감정 예측 분포:")
    print(df['sentiment'].value_counts())

    return df

In [ ]:
# 모델 로드
model, tokenizer = load_model('./news_bias_model', class_weights)  # 모델 경로 변경
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# CSV 파일 예측
csv_path = "/content/drive/MyDrive/텍스트데이터분석을 위한 딥러닝/팀프로젝트/data/전체1.csv"  # 여기에 실제 CSV 파일 경로를 입력하시면 됩니다
predicted_df = predict_csv_file(csv_path, model, tokenizer, device)

In [33]:
# 저장
predicted_df.to_csv(csv_path, index=False)

# 2번째 시도

In [ ]:
# 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/텍스트데이터분석을 위한 딥러닝/팀프로젝트/data/전체1.csv')  # 파일 경로는 실제 경로에 맞게 수정해주세요

# 정당 레이블 매핑
party_mapping = {'국민의힘': 0, '민주당': 1, '그외': 2}

# 정당 레이블 변환
df['party_label'] = df['party'].map(party_mapping)

# 감정 레이블 매핑
sentiment_mapping = {'긍정': 0, '중립': 1, '부정': 2}

# 감정 레이블 변환
df['sentiment_label'] = df['sentiment'].map(sentiment_mapping)

# NaN 값 처리
df = df.dropna(subset=['title_cleaned', 'content_cleaned', 'party_label', 'sentiment_label'])

# 제목과 본문 결합
df['text'] = df['title_cleaned'] + ' ' + df['content_cleaned']

print(f"전체 데이터 수: {len(df)}")
print("\n정당별 기사 수:")
print(df['party'].value_counts())
print("\n정당 레이블 분포:")
print(df['party_label'].value_counts())
print("\n감정별 기사 수:")
print(df['sentiment'].value_counts())
print("\n감정 레이블 분포:")
print(df['sentiment_label'].value_counts())

In [ ]:
# 데이터 분할
train_texts, val_texts, train_party_labels, val_party_labels, train_sentiment_labels, val_sentiment_labels = train_test_split(
    df['text'].values,
    df['party_label'].values,
    df['sentiment_label'].values,
    test_size=0.2,
    random_state=42
)

# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

# 데이터셋 생성
train_dataset = NewsDataset(train_texts, train_party_labels, train_sentiment_labels, tokenizer)
val_dataset = NewsDataset(val_texts, val_party_labels, val_sentiment_labels, tokenizer)

label_list = [example['party_label'] for example in train_dataset]
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array([0, 1, 2]),  # 0: 국민의힘, 1: 민주당, 2: 그외
    y=np.array(label_list)
)
print("클래스 가중치:", class_weights)

# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화 및 GPU 이동을 분리
model = NewsBiasModel('klue/roberta-base', class_weights=class_weights)
model = model.to(device)

# 커스텀 데이터 콜레이터
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./news_bias_results',  # 디렉토리 이름 변경
    num_train_epochs=15,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=3.3541e-05,
    warmup_ratio=0.1135,
    weight_decay=0.0256,
    logging_dir='./news_bias_logs',  # 로그 디렉토리 이름 변경
    logging_steps=100,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='party_f1',  # 평가 메트릭 이름 변경
    gradient_accumulation_steps=3,  # 그래디언트 누적
    max_grad_norm=0.7155,  # 그래디언트 클리핑
    fp16=True,  # 혼합 정밀도 학습
    label_smoothing_factor=0.0672,  # 레이블 스무딩
    optim='adamw_torch',  # AdamW 옵티마이저 사용
    lr_scheduler_type='cosine'
)

# 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.001)]
)

# 학습 실행
trainer.train()

# 최종 평가
final_metrics = trainer.evaluate()
print("\n최종 평가 결과:")
print(f"정당 분류 F1 점수: {final_metrics['eval_party_f1']:.4f}")
print(f"정당 분류 정확도: {final_metrics['eval_party_accuracy']:.4f}")
print(f"감정 분류 F1 점수: {final_metrics['eval_sentiment_f1']:.4f}")
print(f"감정 분류 정확도: {final_metrics['eval_sentiment_accuracy']:.4f}")

# 베이스라인과 성능 비교
comparison_results = {
    'Model': ['Baseline', 'News Bias BERT'],  # 모델 이름 변경
    'Party F1': [baseline_results['party_f1'], final_metrics['eval_party_f1']],  # 메트릭 이름 변경
    'Party Accuracy': [baseline_results['party_accuracy'], final_metrics['eval_party_accuracy']],  # 메트릭 이름 변경
}

comparison_df = pd.DataFrame(comparison_results)
comparison_df.to_csv('./news_bias_results/model_comparison.csv', index=False)  # 저장 경로 변경
comparison_df.head()

# 성능 향상 시각화
plt.figure(figsize=(12, 6))
metrics = ['Party F1', 'Party Accuracy']  # 메트릭 이름 변경
x = np.arange(len(metrics))
width = 0.35

plt.bar(x - width/2, comparison_df.iloc[0, 1:], width, label='Baseline')
plt.bar(x + width/2, comparison_df.iloc[1, 1:], width, label='News Bias BERT')  # 모델 이름 변경

plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Model Performance Comparison')
plt.xticks(x, metrics)
plt.legend()

plt.savefig('./news_bias_results/performance_comparison.png')  # 저장 경로 변경
plt.show()

# 모델 저장
trainer.save_model('./news_bias_model')  # 저장 경로 변경
tokenizer.save_pretrained('./news_bias_model')  # 저장 경로 변경

# 모델 로드
model, tokenizer = load_model('./news_bias_model', class_weights)  # 모델 경로 변경
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# CSV 파일 예측
csv_path = "/content/drive/MyDrive/텍스트데이터분석을 위한 딥러닝/팀프로젝트/data/전체2.csv"  # 여기에 실제 CSV 파일 경로를 입력하시면 됩니다
predicted_df = predict_csv_file(csv_path, model, tokenizer, device)

# 저장
predicted_df.to_csv(csv_path, index=False)

# 3번째 시도

In [ ]:
# 데이터 로드
df1 = pd.read_csv('/content/drive/MyDrive/텍스트데이터분석을 위한 딥러닝/팀프로젝트/data/전체1.csv')
df2 = pd.read_csv('/content/drive/MyDrive/텍스트데이터분석을 위한 딥러닝/팀프로젝트/data/전체2.csv')
df = pd.concat([df1, df2], ignore_index=True)

# 정당 레이블 매핑
party_mapping = {'국민의힘': 0, '민주당': 1, '그외': 2}

# 정당 레이블 변환
df['party_label'] = df['party'].map(party_mapping)

# 감정 레이블 매핑
sentiment_mapping = {'긍정': 0, '중립': 1, '부정': 2}

# 감정 레이블 변환
df['sentiment_label'] = df['sentiment'].map(sentiment_mapping)

# NaN 값 처리
df = df.dropna(subset=['title_cleaned', 'content_cleaned', 'party_label', 'sentiment_label'])

# 제목과 본문 결합
df['text'] = df['title_cleaned'] + ' ' + df['content_cleaned']

print(f"전체 데이터 수: {len(df)}")
print("\n정당별 기사 수:")
print(df['party'].value_counts())
print("\n정당 레이블 분포:")
print(df['party_label'].value_counts())
print("\n감정별 기사 수:")
print(df['sentiment'].value_counts())
print("\n감정 레이블 분포:")
print(df['sentiment_label'].value_counts())

In [ ]:
# 데이터 분할
train_texts, val_texts, train_party_labels, val_party_labels, train_sentiment_labels, val_sentiment_labels = train_test_split(
    df['text'].values,
    df['party_label'].values,
    df['sentiment_label'].values,
    test_size=0.2,
    random_state=42
)

# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

# 데이터셋 생성
train_dataset = NewsDataset(train_texts, train_party_labels, train_sentiment_labels, tokenizer)
val_dataset = NewsDataset(val_texts, val_party_labels, val_sentiment_labels, tokenizer)

label_list = [example['party_label'] for example in train_dataset]
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array([0, 1, 2]),  # 0: 국민의힘, 1: 민주당, 2: 그외
    y=np.array(label_list)
)
print("클래스 가중치:", class_weights)

# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화 및 GPU 이동을 분리
model = NewsBiasModel('klue/roberta-base', class_weights=class_weights)
model = model.to(device)

# 커스텀 데이터 콜레이터
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./news_bias_results',  # 디렉토리 이름 변경
    num_train_epochs=15,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=3.3541e-05,
    warmup_ratio=0.1135,
    weight_decay=0.0256,
    logging_dir='./news_bias_logs',  # 로그 디렉토리 이름 변경
    logging_steps=100,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='party_f1',  # 평가 메트릭 이름 변경
    gradient_accumulation_steps=3,  # 그래디언트 누적
    max_grad_norm=0.7155,  # 그래디언트 클리핑
    fp16=True,  # 혼합 정밀도 학습
    label_smoothing_factor=0.0672,  # 레이블 스무딩
    optim='adamw_torch',  # AdamW 옵티마이저 사용
    lr_scheduler_type='cosine'
)

# 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.001)]
)

# 학습 실행
trainer.train()

# 최종 평가
final_metrics = trainer.evaluate()
print("\n최종 평가 결과:")
print(f"정당 분류 F1 점수: {final_metrics['eval_party_f1']:.4f}")
print(f"정당 분류 정확도: {final_metrics['eval_party_accuracy']:.4f}")
print(f"감정 분류 F1 점수: {final_metrics['eval_sentiment_f1']:.4f}")
print(f"감정 분류 정확도: {final_metrics['eval_sentiment_accuracy']:.4f}")

# 베이스라인과 성능 비교
comparison_results = {
    'Model': ['Baseline', 'News Bias BERT'],  # 모델 이름 변경
    'Party F1': [baseline_results['party_f1'], final_metrics['eval_party_f1']],  # 메트릭 이름 변경
    'Party Accuracy': [baseline_results['party_accuracy'], final_metrics['eval_party_accuracy']],  # 메트릭 이름 변경
}

comparison_df = pd.DataFrame(comparison_results)
comparison_df.to_csv('./news_bias_results/model_comparison.csv', index=False)  # 저장 경로 변경
comparison_df.head()

# 성능 향상 시각화
plt.figure(figsize=(12, 6))
metrics = ['Party F1', 'Party Accuracy']  # 메트릭 이름 변경
x = np.arange(len(metrics))
width = 0.35

plt.bar(x - width/2, comparison_df.iloc[0, 1:], width, label='Baseline')
plt.bar(x + width/2, comparison_df.iloc[1, 1:], width, label='News Bias BERT')  # 모델 이름 변경

plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Model Performance Comparison')
plt.xticks(x, metrics)
plt.legend()

plt.savefig('./news_bias_results/performance_comparison.png')  # 저장 경로 변경
plt.show()

# 모델 저장
trainer.save_model('./news_bias_model')  # 저장 경로 변경
tokenizer.save_pretrained('./news_bias_model')  # 저장 경로 변경

# 모델 로드
model, tokenizer = load_model('./news_bias_model', class_weights)  # 모델 경로 변경
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# CSV 파일 예측
csv_path = "/content/drive/MyDrive/텍스트데이터분석을 위한 딥러닝/팀프로젝트/data/전체통합_전처리.csv"  # 여기에 실제 CSV 파일 경로를 입력하시면 됩니다
predicted_df = predict_csv_file(csv_path, model, tokenizer, device)

# 저장
predicted_df.to_csv(csv_path, index=False)

# 예측 결과

In [ ]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/텍스트데이터분석을 위한 딥러닝/팀프로젝트/data/전체통합_전처리.csv')  # 파일명에 맞게 수정

# 전체 기사 중 각 정당별 비율
party_counts = df['party'].value_counts(normalize=True) * 100
print("전체 기사 편향성(%)")
print(party_counts)

# 전체 기사 중  각 감정별 비율
sentiment_counts = df['sentiment'].value_counts(normalize=True) * 100
print("전체 기사 감정 분포(%)")
print(sentiment_counts)

In [ ]:
# 언론사별 정당 비율
press_party = df.groupby('press')['party'].value_counts(normalize=True).unstack().fillna(0) * 100
print("언론사별 편향성(%)")
print(press_party)

# 언론사별 감정 비율
press_sentiment = df.groupby('press')['sentiment'].value_counts(normalize=True).unstack().fillna(0) * 100
print("언론사별 감정 분포(%)")
print(press_sentiment)

In [51]:
import matplotlib.pyplot as plt

# 윈도우의 기본 한글 폰트 설정 (예: 'Malgun Gothic')
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스(-) 깨짐 방지

In [ ]:
# 전체 편향성
party_counts.plot(kind='bar')
plt.title('전체 기사 편향성')
plt.ylabel('비율(%)')
plt.show()

In [ ]:
# 전체 감정 분포
sentiment_counts.plot(kind='bar')
plt.title('전체 기사 감정 분포')
plt.ylabel('비율(%)')
plt.show()

In [ ]:
# 언론사별 편향성
press_party.plot(kind='bar', stacked=True, figsize=(10,6))
plt.title('언론사별 기사 편향성')
plt.ylabel('비율(%)')
plt.show()

In [ ]:
# 언론사별 감정 분포
press_sentiment.plot(kind='bar', stacked=True, figsize=(10,6))
plt.title('언론사별 기사 감정 분포')
plt.ylabel('비율(%)')
plt.show()

In [ ]:
train_log = trainer.state.log_history
train_losses = [x['loss'] for x in train_log if 'loss' in x]
eval_losses = [x['eval_loss'] for x in train_log if 'eval_loss' in x]
eval_party_f1 = [x['eval_party_f1'] for x in train_log if 'eval_party_f1' in x]
eval_party_accuracy = [x['eval_party_accuracy'] for x in train_log if 'eval_party_accuracy' in x]
eval_sentiment_f1 = [x['eval_sentiment_f1'] for x in train_log if 'eval_sentiment_f1' in x]
eval_sentiment_accuracy = [x['eval_sentiment_accuracy'] for x in train_log if 'eval_sentiment_accuracy' in x]

plt.figure(figsize=(8, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(eval_losses, label='Validation Loss')
plt.plot(eval_party_f1, label='Party F1')
plt.plot(eval_party_accuracy, label='Party Accuracy')
plt.plot(eval_sentiment_f1, label='Sentiment F1')
plt.plot(eval_sentiment_accuracy, label='Sentiment Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss and Metrics')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()